In [19]:
# Excercise 2


In [20]:
import sys
from pathlib import Path

sys.path.insert(0, "..")


Haal de data op vanuit de functie in de gestructeerde folder indeling.

In [21]:
from src.data import make_dataset
from src.data import data_tools

from typing import Callable, Dict, Iterator, List, Optional, Sequence, Tuple, Union

import numpy as np
import tensorflow as tf
import torch
from loguru import logger


Tensor = torch.Tensor


In [22]:
dataset = make_dataset.get_eeg()


2022-06-12 10:39:43.510 | INFO     | src.data.make_dataset:get_eeg:38 - Data is downloaded to /tmp/.keras/datasets/eeg.


## Exercise 1
Na een verkenning van de data, komt naar voren dat de datset uit 14 dimensies staat.De data wordt eerst ingeladen in chunks wanneer het label gelijk is. Hierbij wordt dus de data aan elkaar geclusterd wanneer het label gelijk blijft. Dit wordt gedaan door middel van de class BaseDataset waarbij ik een functie ingebouwd heb waarbij de data wordt aangepast (process_data). Hierbij kijk ik eerst naar het huidige label (cur_label = index 14) en maak ik 2 lege lijsten aan. 1 lijst waar meerdere observaties (EEG_list) aan elkaar worden opgeslagen en 1 lijst waar alle chunks worden samengevoegd (chunks). Waarbij ik gebruik maak van een forloop en kijk telkens of het label gelijk is aan het huidige label en hierna pak ik alle indexen behalve index 14 ( = label). Deze voeg ik toe aan de een lege lijst (EEG_dim) en uiteindelijk aan de lijst buiten de forloop (EEG_list). Dit doe ik ook als het label niet gelijk is aan huidig label. Uiteindelijk voeg ik de lijsten weer bij elkaar in de list chunks.  

In [23]:
dataloader = data_tools.BaseDataset(datapath=dataset)


Hierbij komen 24 chunks naar voren. 

In [24]:
dataloader.__len__()


24

In [25]:
dataloader.__getitem__(0)


(0,
 tensor([[4329.2300, 4009.2300, 4289.2300,  ..., 4280.5098, 4635.8999,
          4393.8501],
         [4324.6201, 4004.6201, 4293.8501,  ..., 4279.4902, 4632.8198,
          4384.1001],
         [4327.6899, 4006.6699, 4295.3799,  ..., 4282.0498, 4628.7202,
          4389.2300],
         ...,
         [4468.2100, 4044.6201, 4305.1299,  ..., 4367.6899, 4833.8501,
          4571.7900],
         [4461.0298, 4041.0300, 4300.0000,  ..., 4365.1299, 4826.6699,
          4558.4600],
         [4452.8198, 4032.3101, 4295.3799,  ..., 4353.3301, 4808.2100,
          4549.2300]]))

In [26]:
x, y = dataloader[0]
x, y.shape


(0, torch.Size([188, 14]))

Hierbij zie je dus dat de eerste chunk uit een sequence lengte van 188 bij 14 dimensies bestaat met label 0. De tweede observatie een sequence lengte van 683 bij 14 dimensies heeft met label 1. Om dit te kunnen batchen kunnen we gebruik maken van een window functie. Deze heb ik een de BaseDataIterator toegevoegd (class BaseDataIterator_wind). Hierbij maak ik gebruik van de class van de BaseDataset en heb hierbij een functie window ingebouwd. Hierbij heb ik als basis de window functie uit les 3 gebruikt en maak ik weer een lege lijst aan. Hierna maak ik weer gebruik van een forloop waarbij ik door de 24 chunks van de dataset ga (len(self.dataset)). Hierbij kijk ik dus per chunk hoelang de sequence lengte is en wordt er gekeken hoevaak de window erdoor heen kan.   

In [27]:
window_data = data_tools.BaseDataIterator_wind(dataloader, 5)


In [28]:
window_data.__len__()


24

In onderstaande uitdraai zie je het eerste item in combinatie met een window van 5, waarbij het tweede gedeelte dus ook met de tweede waarde begint (4468). 

In [29]:
window_data.__getitem__(0)


(0,
 tensor([[[4465.1299, 4041.5400, 4298.4600,  ..., 4358.4600, 4814.3599,
           4566.1499],
          [4468.2100, 4044.6201, 4305.1299,  ..., 4367.6899, 4833.8501,
           4571.7900],
          [4461.0298, 4041.0300, 4300.0000,  ..., 4365.1299, 4826.6699,
           4558.4600],
          [4452.8198, 4032.3101, 4295.3799,  ..., 4353.3301, 4808.2100,
           4549.2300],
          [4329.2300, 4009.2300, 4289.2300,  ..., 4280.5098, 4635.8999,
           4393.8501]],
 
         [[4468.2100, 4044.6201, 4305.1299,  ..., 4367.6899, 4833.8501,
           4571.7900],
          [4461.0298, 4041.0300, 4300.0000,  ..., 4365.1299, 4826.6699,
           4558.4600],
          [4452.8198, 4032.3101, 4295.3799,  ..., 4353.3301, 4808.2100,
           4549.2300],
          [4329.2300, 4009.2300, 4289.2300,  ..., 4280.5098, 4635.8999,
           4393.8501],
          [4324.6201, 4004.6201, 4293.8501,  ..., 4279.4902, 4632.8198,
           4384.1001]],
 
         [[4461.0298, 4041.0300, 4300.00

In [30]:
x, y = window_data[0]
x, y.shape


(0, torch.Size([188, 5, 14]))

## Exercise 2

Kijken naar de eerste observatie. Met een window van 5 is de observatie eingelijk te kort ( moet dan 190 zijn) en wordt dus niet de gehele observatie gewindowed. Daarnaast mag de window ook niet te groot zijn. Om deze reden wordt padding toegevoegd. Deze heb ik in een aparte class geschreven waarbij ik de functie padding gebruik. Hierbij maak ik weer een lege lijst waar ik met een forloop door de chunks (24) heen ga. Hierbij kijk ik telkens hoelang de sequence lengte is en bereken ik het verschil met de windowsize door middel van een modula functie (%). Hierbij kijk ik naar het verschil. Stel dit verschil is niet 0 dan wordt deze door midde lvan de F.pad functie toegevoegd. Als deze gelijk is aan 0 wordt er geen extra padding toegepast. 


In [31]:
data_padding = data_tools.BaseDataIterator_pad(dataloader, 5)


In [32]:
data_padding.__len__()


24

In [33]:
data_padding.__getitem__(0)


(0,
 tensor([[4329.2300, 4009.2300, 4289.2300,  ..., 4280.5098, 4635.8999,
          4393.8501],
         [4324.6201, 4004.6201, 4293.8501,  ..., 4279.4902, 4632.8198,
          4384.1001],
         [4327.6899, 4006.6699, 4295.3799,  ..., 4282.0498, 4628.7202,
          4389.2300],
         ...,
         [4452.8198, 4032.3101, 4295.3799,  ..., 4353.3301, 4808.2100,
          4549.2300],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000]]))

Hierbij zie je dus bij de eerste observatie dat de sequence lengte is uitegebreid met 2 ( naar 190), zodat de windowing van 5 nu wel goed gaat en geen data wordt overslagen. Daarnaast is het nu ook mogelijk om een grotere window te gebruiken, aangezien de observaties met een lager aan sequence lengtes aangevuld worden door middel van de padding. De windowing en padding heb ik gecombineerd in één class, waarbij ook de batchloop is toegevoegd. 

In [34]:
data_ex2 = data_tools.BaseDataIterator_pad_wind(dataloader, 60)


In [35]:
data_ex2.__getitem__(0)


(0,
 tensor([[[4450.7700, 4045.1299, 4289.7402,  ..., 4348.2100, 4774.8701,
           4537.4399],
          [4448.7202, 4042.5601, 4275.3799,  ..., 4339.4902, 4785.6401,
           4542.0498],
          [4454.8701, 4039.4900, 4281.0298,  ..., 4345.6401, 4796.9199,
           4551.7900],
          ...,
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [4329.2300, 4009.2300, 4289.2300,  ..., 4280.5098, 4635.8999,
           4393.8501]],
 
         [[4448.7202, 4042.5601, 4275.3799,  ..., 4339.4902, 4785.6401,
           4542.0498],
          [4454.8701, 4039.4900, 4281.0298,  ..., 4345.6401, 4796.9199,
           4551.7900],
          [4465.1299, 4041.5400, 4298.4600,  ..., 4358.4600, 4814.3599,
           4566.1499],
          ...,
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [43

In [36]:
x, y = data_ex2[0]
x, y.shape


(0, torch.Size([240, 60, 14]))